In [1]:
# Syed Rakin Ahmed - 2023-08-22

import os, csv, json, cv2, random, time, sys, shutil
import numpy as np
import pandas as pd

import multiprocessing
from joblib import Parallel, delayed

from tqdm import tqdm
from matplotlib import pyplot as plt
from zipfile import ZipFile
from PIL import Image, ImageDraw

import torch

print(sys.version)
print(torch.__version__)

3.8.8 (default, Feb 24 2021, 21:46:12) 
[GCC 7.3.0]
1.9.0a0+2ecb2c7


In [1]:
# Syed Rakin Ahmed - 2023-08-22

# Get all dysis images into one folder

In [2]:
# start = time.time()

# base_dir = '/mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/data/dysis/registered_images/'
# store_dir = '/mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/data/dysis/compiled/'

# for subdir in os.listdir(base_dir):
#     for item in os.listdir(os.path.join(base_dir, subdir)):
#         shutil.copy2(os.path.join(base_dir, subdir, item), store_dir)
        
# end = time.time()
# print(end - start)

# Run OD inference on dysis dataset

In [3]:
# python /mnt/2015P002510/rakin/object_detection_repos/yolov5/detect.py \
# --weights /mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/models/detection/V2/train_100ep_yolov5s_5study_liger_cambodia_80_10_10.pt \
# --source /mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/data/dysis/compiled/ \
# --save-txt --save-conf \
# --project /mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/predictions/20230822_detector_predictions_v2/ \
# --conf-thres 0 --max-det 1

# Process dysis data

In [4]:
img_src_dir = '/mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/data/dysis/compiled/'
im_list = os.listdir(img_src_dir)
print(len(im_list))
print(im_list[0])

4098
00017_img000.bmp


In [5]:
data_dict = {}

data_dict['image_fname'] = im_list

for item in data_dict.keys():
    print(f"{item} - {data_dict[item][0]}")

image_fname - 00017_img000.bmp


In [6]:
df_orig = pd.DataFrame.from_dict(data_dict)
df_orig

,image_fname
0,00017_img000.bmp
1,00017_img007.bmp
2,00017_img014.bmp
3,00017_img021.bmp
4,00017_img028.bmp
...,...
4093,05620_img084.jpg
4094,05620_img094.jpg
4095,05620_img104.jpg
4096,05620_img114.jpg


In [7]:
len('/mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/predictions/20230822_detector_predictions_v2/images/')

135

In [8]:
def get_OD_output_dict(img_store_dir, txt_store_dir):
    image_files = [os.path.join(img_store_dir, img_file) for img_file in os.listdir(img_store_dir)]
    annotation_files = [image_file.replace("images", "labels").replace(image_file[(image_file.find(".")):], '.txt') for image_file in image_files]
    
    record_list = []
    
    # Images successfully detected by object detection network
    for image_file, annotation_file in zip(image_files, annotation_files):
        record = {}
        with open(annotation_file, "r") as file:
            annotation_list = file.read().split("\n")[:-1]
        #
        annotation_list = [x.split(" ") for x in annotation_list]
        annotation_list = [[float(y) for y in x ] for x in annotation_list]
        #
        img = Image.open(image_file)
        annotations = np.array(annotation_list)
        w, h = img.size
        #
        transformed_annotations = np.copy(annotations)
        transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
        transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h
        #
        transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
        transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
        transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
        transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
        #
        for ann in transformed_annotations:
            obj_cls, x1, y1, x2, y2, conf = ann
        #    
        record['image_path'] = image_file
        record['image'] = image_file[135:]        
        record['x1'] = round(x1)
        record['y1'] = round(y1)
        record['x2'] = round(x2)
        record['y2'] = round(y2)
        record['det_conf'] = conf
        record['dataset'] = 'test'
        #
        record_list.append(record)

    return record_list

In [9]:
img_store_dir = '/mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/predictions/20230822_detector_predictions_v2/images/'
txt_store_dir = '/mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/predictions/20230822_detector_predictions_v2/labels/'
df_list = get_OD_output_dict(img_store_dir, txt_store_dir)

In [10]:
keys = df_list[0].keys()

with open('/mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/predictions/20230822_detector_predictions_v2/dysis_bb_V2.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(df_list)
    
saved_df = pd.read_csv('/mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/predictions/20230822_detector_predictions_v2/dysis_bb_V2.csv')
saved_df.to_csv('/mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/predictions/20230822_detector_predictions_v2/dysis_bb_V2.csv', index=True, index_label=None)

In [11]:
df_bb = pd.read_csv('/mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/predictions/20230822_detector_predictions_v2/dysis_bb_V2.csv')
df_bb

,Unnamed: 0,image_path,image,x1,y1,x2,y2,det_conf,dataset
0,0,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,00017_img000.bmp,2,1,1020,762,0.893247,test
1,1,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,00017_img007.bmp,33,1,1021,654,0.931311,test
2,2,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,00017_img014.bmp,36,2,1023,652,0.932327,test
3,3,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,00017_img021.bmp,36,2,1018,650,0.938566,test
4,4,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,00017_img028.bmp,27,2,1012,645,0.905220,test
...,...,...,...,...,...,...,...,...,...
4093,4093,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,05620_img084.jpg,76,332,1443,1200,0.792751,test
4094,4094,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,05620_img094.jpg,71,344,1442,1200,0.792758,test
4095,4095,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,05620_img104.jpg,59,324,1437,1199,0.770394,test
4096,4096,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,05620_img114.jpg,76,368,1444,1198,0.778058,test


In [12]:
cols = ['image_path','image','x1','x2','y1','y2','det_conf','dataset']
df_bb = df_bb[cols]

In [13]:
df_bb

,image_path,image,x1,x2,y1,y2,det_conf,dataset
0,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,00017_img000.bmp,2,1020,1,762,0.893247,test
1,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,00017_img007.bmp,33,1021,1,654,0.931311,test
2,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,00017_img014.bmp,36,1023,2,652,0.932327,test
3,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,00017_img021.bmp,36,1018,2,650,0.938566,test
4,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,00017_img028.bmp,27,1012,2,645,0.905220,test
...,...,...,...,...,...,...,...,...
4093,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,05620_img084.jpg,76,1443,332,1200,0.792751,test
4094,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,05620_img094.jpg,71,1442,344,1200,0.792758,test
4095,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,05620_img104.jpg,59,1437,324,1199,0.770394,test
4096,/mnt/2015P002510/rakin/cervical_cancer/cervix_...,05620_img114.jpg,76,1444,368,1198,0.778058,test


In [14]:
df_bb.to_csv('/mnt/2015P002510/rakin/cervical_cancer/cervix_classification/dysis_lesion_detector/predictions/20230822_detector_predictions_v2/dysis_bb_V2.csv', index=True, index_label=None)
